# **HTML Duyarlı Özyineli İşleme ile Türkçeden İngilizceye JSON Arayüz Çevirisinin Güncel Dil Modelleriyle Karşılaştırmalı İncelemesi**

Bu çalışma kapsamında, farklı açık kaynaklı dil modelleri ve çeviri yöntemleri kullanılarak JSON tabanlı kullanıcı arayüzü metinlerinin çok dilli çevirisi gerçekleştirildi. Özellikle HTML içeren metinlerin yapısının korunması amacıyla placeholder-temelli ve rekürsif metin işleme teknikleri uygulandı. Denenen modeller arasında Facebook’un NLLB-200 ve M2M100 serisi, Helsinki-NLP’nin Opus-MT modeli, Google’un mT5-large modeli, Argos Translate kütüphanesi ve daha büyük ölçekli LLaMAX ve Tower 13B gibi son teknoloji büyük dil modelleri yer almaktadır. Modellerin performansları, destekledikleri diller ve işlem süreleri göz önüne alınarak karşılaştırmalı analizler yapıldı. Bu defter, gerçek dünya uygulamalarında kullanıcı arayüzü metinlerinin çok dilli desteklenmesi için farklı yaklaşımların uygulanabilirliğini ve verimliliğini değerlendirmek amacıyla hazırlanmıştır.

# JSON UI Translation with NLLB-200 (3.3B) using Placeholder-Based HTML-Aware Text Replacement


Bu Python kodu, Facebook'un `nllb-200-3.3B` modelini kullanarak JSON formatındaki çok katmanlı arayüz metinlerini Türkçeden İngilizceye çevirir. Kod, JSON objelerini rekurzif olarak işleyerek hem sözlük (dict) hem liste (list) yapılarında bulunan tüm metin değerlerini ayıklar ve transformers kütüphanesindeki NLLB modeli ile GPU hızlandırmalı şekilde çeviri yapar. Metin içinde gömülü olan HTML etiketleri, regex ile tespit edilip yer tutucularla maskelenir, böylece modelin bu yapıları bozması engellenir. Çeviri sırasında, NLLB modelinin zorunlu başlangıç tokenı (`forced_bos_token_id`) hedef dil koduna uygun şekilde atanır. Sonuçta, yapısal bütünlük korunarak, sadece içerik metinleri hedef dile yüksek doğrulukla çevrilmiş yeni bir JSON çıktısı oluşturulur. Bu yaklaşım, çok dilli yazılım arayüzü lokalizasyon projelerinde otomatik, yapı korumalı ve yüksek kaliteli çeviri sağlamaya yönelik endüstriyel düzeyde bir çözümdür.

In [1]:
pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [2]:
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
# Model Yükleme
model_name = "facebook/nllb-200-3.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

src_lang = "tur_Latn"
tgt_lang = "eng_Latn"

tokenizer.src_lang = src_lang

# Model yükleme süresi : 1 dakika 41 saniye

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/8.55G [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/6.93G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [4]:
# Metin Çeviri Fonksiyonu
def translate_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)
    inputs["forced_bos_token_id"] = forced_bos_token_id
    with torch.no_grad():
        translated_tokens = model.generate(**inputs, max_length=512)
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

In [5]:
def translate_text_with_html(text):
    # HTML yoksa direkt çevir
    if '<' not in text:
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(**inputs, max_length=512)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # HTML etiketleri ve metinleri ayır
    parts = re.split(r'(<[^>]*>)', text)
    translated_parts = []

    for part in parts:
        if not part:  # Boş kısım
            continue
        elif part.startswith('<') and part.endswith('>'):
            # HTML etiketi - olduğu gibi bırak
            translated_parts.append(part)
        else:
            # Metin kısmı - çevir (sadece anlamlı metin varsa)
            clean_text = part.strip()
            if clean_text:
                inputs = tokenizer(clean_text, return_tensors="pt", truncation=True).to(device)
                outputs = model.generate(**inputs, max_length=512)
                translated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

                # Orijinal boşlukları koru
                if part.startswith(' '):
                    translated = ' ' + translated
                if part.endswith(' '):
                    translated = translated + ' '

                translated_parts.append(translated)
            else:
                # Sadece boşluk varsa olduğu gibi bırak
                translated_parts.append(part)

    return ''.join(translated_parts)

In [6]:
# JSON'ı Rekürsif Olarak Çevir
# JSON içindeki tüm metin alanlarını iç içe yapılar dahil olmak üzere dolaşarak (recursive) çevirir.
def translate_json(data):
    if isinstance(data, dict):
        result = {}
        for k, v in data.items():
            if isinstance(v, str) and k.lower() == v.lower():
                result[k] = v  # Anahtar ve değer aynıysa çevirme
            else:
                result[k] = translate_json(v)
        return result
    elif isinstance(data, list):
        return [translate_json(item) for item in data]
    elif isinstance(data, str):
        return translate_text_with_html(data)
    else:
        return data

In [7]:
input_path = "tr.json"
output_path = "translated_eng.json"

In [8]:
# Dosya oku, Çevir, Kaydet
with open(input_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

translated_data = translate_json(original_data)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"Çeviri tamamlandı.")

# Çeviri süresi : 1 dakika 58 saniye
# Kullanılan GPU A100
# Kullanılan Sistem RAM : 5.8 GB
# Kullanılan GPU RAM : 13 GB

Çeviri tamamlandı.


# JSON UI Translation with M2M100 (1.2B) using Placeholder-Based HTML-Aware Text Replacement

Bu Python kodu, Facebook'un `m2m100_1.2B` modelini kullanarak JSON formatındaki çok katmanlı arayüz metinlerini Türkçeden İngilizceye çevirir. Kod, JSON objelerini rekurzif olarak işleyerek hem sözlük (dict) hem liste (list) yapılarında bulunan tüm metin değerlerini ayıklar ve transformers kütüphanesindeki NLLB modeli ile GPU hızlandırmalı şekilde çeviri yapar. Metin içinde gömülü olan HTML etiketleri, regex ile tespit edilip yer tutucularla maskelenir, böylece modelin bu yapıları bozması engellenir. Çeviri sırasında, NLLB modelinin zorunlu başlangıç tokenı (`forced_bos_token_id`) hedef dil koduna uygun şekilde atanır. Sonuçta, yapısal bütünlük korunarak, sadece içerik metinleri hedef dile yüksek doğrulukla çevrilmiş yeni bir JSON çıktısı oluşturulur. Bu yaklaşım, çok dilli yazılım arayüzü lokalizasyon projelerinde otomatik, yapı korumalı ve yüksek kaliteli çeviri sağlamaya yönelik endüstriyel düzeyde bir çözümdür.

In [ ]:
pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [ ]:
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
# Model Yükleme
model_name = "facebook/m2m100_1.2B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

src_lang = "tr"
tgt_lang = "en"

tokenizer.src_lang = src_lang

# Model yükleme süresi : 33 saniye

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [ ]:
# Metin Çeviri Fonksiyonu
def translate_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    forced_bos_token_id = tokenizer.get_lang_id(tgt_lang)
    inputs["forced_bos_token_id"] = forced_bos_token_id
    with torch.no_grad():
        translated_tokens = model.generate(**inputs, max_length=512)
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

In [ ]:
def translate_text_with_html(text):
    # HTML yoksa direkt çevir
    if '<' not in text:
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(**inputs, max_length=512)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # HTML etiketleri ve metinleri ayır
    parts = re.split(r'(<[^>]*>)', text)
    translated_parts = []

    for part in parts:
        if not part:  # Boş kısım
            continue
        elif part.startswith('<') and part.endswith('>'):
            # HTML etiketi - olduğu gibi bırak
            translated_parts.append(part)
        else:
            # Metin kısmı - çevir (sadece anlamlı metin varsa)
            clean_text = part.strip()
            if clean_text:
                inputs = tokenizer(clean_text, return_tensors="pt", truncation=True).to(device)
                outputs = model.generate(**inputs, max_length=512)
                translated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

                # Orijinal boşlukları koru
                if part.startswith(' '):
                    translated = ' ' + translated
                if part.endswith(' '):
                    translated = translated + ' '

                translated_parts.append(translated)
            else:
                # Sadece boşluk varsa olduğu gibi bırak
                translated_parts.append(part)

    return ''.join(translated_parts)

In [ ]:
# JSON'ı Rekürsif Olarak Çevir
# JSON içindeki tüm metin alanlarını iç içe yapılar dahil olmak üzere dolaşarak (recursive) çevirir.
def translate_json(data):
    if isinstance(data, dict):
        result = {}
        for k, v in data.items():
            if isinstance(v, str) and k.lower() == v.lower():
                result[k] = v  # Anahtar ve değer aynıysa çevirme
            else:
                result[k] = translate_json(v)
        return result
    elif isinstance(data, list):
        return [translate_json(item) for item in data]
    elif isinstance(data, str):
        return translate_text_with_html(data)
    else:
        return data

In [ ]:
input_path = "tr.json"
output_path = "translated_eng_2.json"

In [ ]:
# Dosya oku, Çevir, Kaydet
with open(input_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

translated_data = translate_json(original_data)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"Çeviri tamamlandı.")

# Çeviri süresi : 2 dakika 14 saniye
# Kullanılan GPU A100
# Kullanılan Sistem RAM : 5.6 GB
# Kullanılan GPU RAM : 5.2 GB

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Çeviri tamamlandı.


# Lightweight JSON UI Translation Using Argos Translate

Bu çalışma, Türkçe JSON dosyalarındaki kullanıcı arayüz metinlerini hafif ve yerel çalışabilen `Argos Translate (MarianMT tabanlı)` modeli ile İngilizceye çevirmek için tasarlanmıştır. HTML etiketleri korunarak metinler bozulmadan çevrilir. Kod, JSON yapısını rekürsif olarak tarayarak tüm string alanları çevirir ve sonucu translated.json dosyasına kaydeder. GPU gerekmeden sadece CPU ile çalışabilir ve düşük kaynaklı sistemlerde hızlı sonuç verir.

In [ ]:
pip install argostranslate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 39.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:

In [ ]:
import json
import re
import os
import argostranslate.package, argostranslate.translate

In [ ]:
# Argos Translate paketini yükle (ilk seferde gerekebilir)
def setup_argos_translate():
    packages = argostranslate.package.get_available_packages()
    tr_en = list(filter(lambda p: p.from_code == "tr" and p.to_code == "en", packages))
    if tr_en:
        argostranslate.package.install_from_path(tr_en[0].download())

In [ ]:
# İlk yükleme gerekiyorsa aktif et
setup_argos_translate()

# Paketin Yüklenme SÜresi : 3 saniye

In [ ]:
# Çeviri modeli seçimi
installed_languages = argostranslate.translate.get_installed_languages()
from_lang = list(filter(lambda x: x.code == "tr", installed_languages))[0]
to_lang = list(filter(lambda x: x.code == "en", installed_languages))[0]
translator = from_lang.get_translation(to_lang)

In [ ]:
def translate_text_with_html(text):
    # HTML yoksa direkt çevir
    if '<' not in text:
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(**inputs, max_length=512)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # HTML etiketleri ve metinleri ayır
    parts = re.split(r'(<[^>]*>)', text)
    translated_parts = []

    for part in parts:
        if not part:  # Boş kısım
            continue
        elif part.startswith('<') and part.endswith('>'):
            # HTML etiketi - olduğu gibi bırak
            translated_parts.append(part)
        else:
            # Metin kısmı - çevir (sadece anlamlı metin varsa)
            clean_text = part.strip()
            if clean_text:
                inputs = tokenizer(clean_text, return_tensors="pt", truncation=True).to(device)
                outputs = model.generate(**inputs, max_length=512)
                translated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

                # Orijinal boşlukları koru
                if part.startswith(' '):
                    translated = ' ' + translated
                if part.endswith(' '):
                    translated = translated + ' '

                translated_parts.append(translated)
            else:
                # Sadece boşluk varsa olduğu gibi bırak
                translated_parts.append(part)

    return ''.join(translated_parts)

In [ ]:
# JSON'ı rekürsif olarak çevir
def translate_json(data):
    if isinstance(data, dict):
        return {k: translate_json(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [translate_json(item) for item in data]
    elif isinstance(data, str):
        return translate_text_with_html(data)
    else:
        return data

In [ ]:
input_path = "tr.json"
output_path = "translated_eng_3.json"

In [ ]:
# JSON dosyasını çevir
with open(input_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

translated_data = translate_json(original_data)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"Çeviri tamamlandı")

# Çeviri süresi : 28 saniye
# Kullanılan GPU A100
# Kullanılan Sistem RAM : 3.9 GB
# Kullanılan GPU RAM : 0 GB (CPU kullanıldı GPU gerekmiyor)

Çeviri tamamlandı


# JSON UI Translation with M2M100 (12B) using Placeholder-Based HTML-Aware Text Replacement

Bu Python kodu, Facebook'un `m2m100_12B` modelini kullanarak JSON formatındaki çok katmanlı arayüz metinlerini Türkçeden İngilizceye çevirir. Kod, JSON objelerini rekurzif olarak işleyerek hem sözlük (dict) hem liste (list) yapılarında bulunan tüm metin değerlerini ayıklar ve transformers kütüphanesindeki NLLB modeli ile GPU hızlandırmalı şekilde çeviri yapar. Metin içinde gömülü olan HTML etiketleri, regex ile tespit edilip yer tutucularla maskelenir, böylece modelin bu yapıları bozması engellenir. Çeviri sırasında, NLLB modelinin zorunlu başlangıç tokenı (`forced_bos_token_id`) hedef dil koduna uygun şekilde atanır. Sonuçta, yapısal bütünlük korunarak, sadece içerik metinleri hedef dile yüksek doğrulukla çevrilmiş yeni bir JSON çıktısı oluşturulur. Bu yaklaşım, çok dilli yazılım arayüzü lokalizasyon projelerinde otomatik, yapı korumalı ve yüksek kaliteli çeviri sağlamaya yönelik endüstriyel düzeyde bir çözümdür.

In [ ]:
pip install torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import json
import re
import torch
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration

In [ ]:
# Model Yükleme

model_name = "facebook/m2m100-12B-last-ckpt"

tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",          # GPU belleği için otomatik dağıtım
    torch_dtype=torch.float16   # A100 için uygun ve verimli
)

tokenizer.src_lang = "tr"
tgt_lang = "en"

# Model yükleme süresi : 18 dakika

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/47.2G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.2G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
# Metin Çeviri Fonksiyonu
def translate_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    forced_bos_token_id = tokenizer.get_lang_id(tgt_lang)
    inputs["forced_bos_token_id"] = forced_bos_token_id
    with torch.no_grad():
        translated_tokens = model.generate(**inputs, max_length=512)
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

In [ ]:
def translate_text_with_html(text):
    # HTML yoksa direkt çevir
    if '<' not in text:
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(**inputs, max_length=512)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # HTML etiketleri ve metinleri ayır
    parts = re.split(r'(<[^>]*>)', text)
    translated_parts = []

    for part in parts:
        if not part:  # Boş kısım
            continue
        elif part.startswith('<') and part.endswith('>'):
            # HTML etiketi - olduğu gibi bırak
            translated_parts.append(part)
        else:
            # Metin kısmı - çevir (sadece anlamlı metin varsa)
            clean_text = part.strip()
            if clean_text:
                inputs = tokenizer(clean_text, return_tensors="pt", truncation=True).to(device)
                outputs = model.generate(**inputs, max_length=512)
                translated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

                # Orijinal boşlukları koru
                if part.startswith(' '):
                    translated = ' ' + translated
                if part.endswith(' '):
                    translated = translated + ' '

                translated_parts.append(translated)
            else:
                # Sadece boşluk varsa olduğu gibi bırak
                translated_parts.append(part)

    return ''.join(translated_parts)

In [ ]:
# JSON'ı Rekürsif Olarak Çevir
# JSON içindeki tüm metin alanlarını iç içe yapılar dahil olmak üzere dolaşarak (recursive) çevirir.
def translate_json(data):
    if isinstance(data, dict):
        result = {}
        for k, v in data.items():
            if isinstance(v, str) and k.lower() == v.lower():
                result[k] = v  # Anahtar ve değer aynıysa çevirme
            else:
                result[k] = translate_json(v)
        return result
    elif isinstance(data, list):
        return [translate_json(item) for item in data]
    elif isinstance(data, str):
        return translate_text_with_html(data)
    else:
        return data

In [ ]:
input_path = "tr.json"
output_path = "translated_eng_6.json"

In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Dosya oku, Çevir, Kaydet
with open(input_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

translated_data = translate_json(original_data)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"Çeviri tamamlandı.")

# Çeviri süresi : 1 dakika 57 saniye
# Kullanılan GPU A100
# Kullanılan Sistem RAM : 7.4 GB
# Kullanılan GPU RAM : 28.4 GB

Çeviri tamamlandı.


# JSON Translation with HTML Preservation Using Helsinki-NLP Opus-MT (Turkish to English)

Bu kod, verilen JSON dosyasındaki tüm metinleri Türkçe'den İngilizce'ye Helsinki-NLP'nin opus-mt-tr-en modeli ile çevirir. HTML etiketlerini bozmamak için öncelikle metindeki HTML taglerini özel yer tutucularla değiştirir, ardından metni çevirir, son olarak yer tutucuları eski haline getirir. Bu sayede çeviri sırasında HTML yapısı korunur.

In [1]:
pip install transformers sentencepiece torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [2]:
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
# Model ve tokenizer yükle
model_name = "Helsinki-NLP/opus-mt-tr-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Model Yükleme Süresi : 26 saniye

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/840k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

In [8]:
# HTML etiketlerini bozmadan çeviri için yardımcı fonksiyon
"""
def translate_text_with_html(text):
    # HTML etiketlerini bul ve yer tutucu ile değiştir
    tags = re.findall(r"<[^>]+>", text)
    placeholder_text = text

    for i, tag in enumerate(tags):
        placeholder_text = placeholder_text.replace(tag, f"[TAG{i}]")

    # Çeviri için tokenizasyon ve model çıkışı
    inputs = tokenizer(placeholder_text, return_tensors="pt", truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=512)
    translated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Yer tutucuları tekrar orijinal HTML etiketleriyle değiştir
    for i, tag in enumerate(tags):
        translated = translated.replace(f"[TAG{i}]", tag)

    return translated
"""

def translate_text_with_html(text):
    # HTML yoksa direkt çevir
    if '<' not in text:
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(**inputs, max_length=512)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # HTML etiketleri ve metinleri ayır
    parts = re.split(r'(<[^>]*>)', text)
    translated_parts = []

    for part in parts:
        if not part:  # Boş kısım
            continue
        elif part.startswith('<') and part.endswith('>'):
            # HTML etiketi - olduğu gibi bırak
            translated_parts.append(part)
        else:
            # Metin kısmı - çevir (sadece anlamlı metin varsa)
            clean_text = part.strip()
            if clean_text:
                inputs = tokenizer(clean_text, return_tensors="pt", truncation=True).to(device)
                outputs = model.generate(**inputs, max_length=512)
                translated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

                # Orijinal boşlukları koru
                if part.startswith(' '):
                    translated = ' ' + translated
                if part.endswith(' '):
                    translated = translated + ' '

                translated_parts.append(translated)
            else:
                # Sadece boşluk varsa olduğu gibi bırak
                translated_parts.append(part)

    return ''.join(translated_parts)

In [9]:
# JSON'u rekürsif olarak çeviren fonksiyon
def translate_json(data):
    if isinstance(data, dict):
        return {k: translate_json(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [translate_json(i) for i in data]
    elif isinstance(data, str):
        return translate_text_with_html(data)
    else:
        return data

In [10]:
input_path = "tr.json"
output_path = "translated.json"

In [11]:
# JSON dosyasını oku, çevir ve kaydet
with open(input_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

translated_data = translate_json(original_data)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"Çeviri tamamlandı.")

# Çeviri süresi : 39 saniye
# Kullanılan GPU A100
# Kullanılan Sistem RAM : 0.2 GB
# Kullanılan GPU RAM : 0 GB (CPU kullanıyor model)

Çeviri tamamlandı.


# ××× JSON UI Translation with google/mt5-large (No Fine-Tuning) using Placeholder-Based HTML-Aware Recursive Text Processing

Bu Colab notebook'u, Google'ın genel amaçlı çok dilli dil modeli `google/mt5-large` kullanarak Türkçe JSON dosyalarındaki metinleri İngilizceye çevirmek üzere tasarlanmıştır. Model, T5 mimarisinin çok dilli versiyonudur ve doğrudan çeviri için özel olarak optimize edilmemiştir; bu nedenle çeviri girişine özel bir prompt (örneğin, "translate Turkish to English:") eklenerek kullanılmaktadır. Kod, JSON içindeki tüm metinleri rekurziv olarak işler ve HTML etiketlerini koruyarak metinlerin yapısını bozmadan çeviri sağlar. GPU veya CPU’da çalışabilir; ancak fine-tune edilmediği için özellikle uzun ve karmaşık metinlerde çeviri kalitesi sınırlı olabilir. Fine-tune edilerek modelin çeviri doğruluğu ve bağlam anlama yeteneği önemli ölçüde artırılabilir. Bu yaklaşım, fine-tune gerektirmeyen hızlı prototip geliştirme için uygundur.

In [ ]:
pip install transformers sentencepiece torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [ ]:
import json
import re
import torch
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

In [ ]:
# Model ve tokenizer yükle
model_name = "google/mt5-large"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Model Yükleme Süresi : 37 saniye

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def translate_text_with_html(text):
    # HTML yoksa direkt çevir
    if '<' not in text:
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(**inputs, max_length=512)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # HTML etiketleri ve metinleri ayır
    parts = re.split(r'(<[^>]*>)', text)
    translated_parts = []

    for part in parts:
        if not part:  # Boş kısım
            continue
        elif part.startswith('<') and part.endswith('>'):
            # HTML etiketi - olduğu gibi bırak
            translated_parts.append(part)
        else:
            # Metin kısmı - çevir (sadece anlamlı metin varsa)
            clean_text = part.strip()
            if clean_text:
                inputs = tokenizer(clean_text, return_tensors="pt", truncation=True).to(device)
                outputs = model.generate(**inputs, max_length=512)
                translated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

                # Orijinal boşlukları koru
                if part.startswith(' '):
                    translated = ' ' + translated
                if part.endswith(' '):
                    translated = translated + ' '

                translated_parts.append(translated)
            else:
                # Sadece boşluk varsa olduğu gibi bırak
                translated_parts.append(part)

    return ''.join(translated_parts)

In [ ]:
# JSON'u rekürsif çevirme fonksiyonu
def translate_json(data):
    if isinstance(data, dict):
        return {k: translate_json(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [translate_json(item) for item in data]
    elif isinstance(data, str):
        return translate_text_with_html(data)
    else:
        return data

In [ ]:
input_path = "tr.json"
output_path = "translated_eng_4.json"

In [ ]:
# JSON dosyasını oku ve çevir
with open(input_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

translated_data = translate_json(original_data)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"Çeviri tamamlandı.")

# Çeviri süresi : +++ 38 dakika ( yaklaşık 40 dakika olmasına rağmen sona ermedi bundan dolayı durdurdum.)
# Kullanılan GPU A100
# Kullanılan Sistem RAM : 9.2 GB
# Kullanılan GPU RAM : 5.3 GB

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-7-2613692193.py", line 5, in <cell line: 0>
    translated_data = translate_json(original_data)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-5-3477616038.py", line 4, in translate_json
    return {k: translate_json(v) for k, v in data.items()}
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-5-3477616038.py", line 4, in <dictcomp>
    return {k: translate_json(v) for k, v in data.items()}
               ^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-5-3477616038.py", line 4, in translate_json
    return {k: translate_json(v) for k, v in data.items()}
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-5-3477616038.py", line 4, in <dictcomp>
    return {k: translate_j

TypeError: object of type 'NoneType' has no len()

# ××× JSON UI Translation with LLaMAX (13B) Using HTML-Aware Placeholder-Based Recursive Text Replacement

Bu kod, LLaMA tabanlı çok dilli bir model olan `LLaMAX`’ı kullanarak JSON içindeki Türkçe arayüz metinlerini İngilizce’ye çevirir. HTML etiketlerini bozmamak için yer tutucu (placeholder) yöntemi uygulanır. Model, prompt tabanlı metin üretimi yapar ve güçlü GPU’larda (örneğin A100) etkin çalışacak şekilde optimize edilmiştir. Rekürsif JSON işleme sayesinde çok katmanlı yapılar da eksiksiz çevrilir. Fine-tune edilmemiş haliyle de genel çeviri kalitesi iyidir, ancak domain’e özgü iyileştirmeler fine-tune ile yapılabilir.

In [ ]:
pip install transformers accelerate sentencepiece torch --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# Model ve tokenizer yükleme
model_name = "UKPLab/dara-llama-2-13b"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,  # A100 için uygun
    cache_dir="cache"

)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Model yükleme süresi :

In [ ]:
def translate_text_with_html(text):
    # HTML yoksa direkt çevir
    if '<' not in text:
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(**inputs, max_length=512)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # HTML etiketleri ve metinleri ayır
    parts = re.split(r'(<[^>]*>)', text)
    translated_parts = []

    for part in parts:
        if not part:  # Boş kısım
            continue
        elif part.startswith('<') and part.endswith('>'):
            # HTML etiketi - olduğu gibi bırak
            translated_parts.append(part)
        else:
            # Metin kısmı - çevir (sadece anlamlı metin varsa)
            clean_text = part.strip()
            if clean_text:
                inputs = tokenizer(clean_text, return_tensors="pt", truncation=True).to(device)
                outputs = model.generate(**inputs, max_length=512)
                translated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

                # Orijinal boşlukları koru
                if part.startswith(' '):
                    translated = ' ' + translated
                if part.endswith(' '):
                    translated = translated + ' '

                translated_parts.append(translated)
            else:
                # Sadece boşluk varsa olduğu gibi bırak
                translated_parts.append(part)

    return ''.join(translated_parts)

In [ ]:
# JSON içindeki tüm metinleri rekürsif olarak çevir
def translate_json(data):
    if isinstance(data, dict):
        return {k: translate_json(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [translate_json(item) for item in data]
    elif isinstance(data, str):
        return translate_text_with_html(data)
    else:
        return data

In [ ]:
input_path = "tr.json"
output_path = "translated_eng_8.json"

In [ ]:
# JSON dosyasını oku, çevir ve kaydet
with open(input_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

translated_data = translate_json(original_data)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"Çeviri tamamlandı")

# Çeviri süresi : 1 dakika 58 saniye
# Kullanılan GPU A100
# Kullanılan Sistem RAM : 5.8 GB
# Kullanılan GPU RAM : 13 GB

# ××× JSON UI Translation with Tower 13B Using HTML Placeholder-Aware Recursive Processing  

Bu çalışma, Unbabel tarafından geliştirilen `Tower 13B` adlı çok dilli büyük dil modeli kullanılarak JSON formatındaki kullanıcı arayüzü metinlerinin Türkçe'den İngilizce'ye çevrilmesini sağlar. Kod, metin içerisindeki HTML etiketlerini bozmadan korumak için placeholder tabanlı bir yöntem uygular. Çeviri işlemi, iç içe geçmiş JSON yapılarında bulunan tüm metinlere rekürsif olarak uygulanır. Model, prompt tabanlı çeviri yapısı sayesinde domain odaklı metinlerde de iyi performans gösterir. A100 gibi güçlü GPU'larda verimli şekilde çalışacak şekilde optimize edilmiştir. Fine-tune işlemi olmadan da başarılı sonuçlar verir, ancak spesifik uygulamalar için ince ayar yapılabilir.

In [ ]:
pip install transformers accelerate sentencepiece torch --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 128.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# Model ve tokenizer yükleme
model_name = "Unbabel/TowerInstruct-13B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16  # A100 GPU için uygun
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Model yükleme süresi :

In [ ]:
def translate_text_with_html(text):
    # HTML yoksa direkt çevir
    if '<' not in text:
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(device)
        outputs = model.generate(**inputs, max_length=512)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # HTML etiketleri ve metinleri ayır
    parts = re.split(r'(<[^>]*>)', text)
    translated_parts = []

    for part in parts:
        if not part:  # Boş kısım
            continue
        elif part.startswith('<') and part.endswith('>'):
            # HTML etiketi - olduğu gibi bırak
            translated_parts.append(part)
        else:
            # Metin kısmı - çevir (sadece anlamlı metin varsa)
            clean_text = part.strip()
            if clean_text:
                inputs = tokenizer(clean_text, return_tensors="pt", truncation=True).to(device)
                outputs = model.generate(**inputs, max_length=512)
                translated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

                # Orijinal boşlukları koru
                if part.startswith(' '):
                    translated = ' ' + translated
                if part.endswith(' '):
                    translated = translated + ' '

                translated_parts.append(translated)
            else:
                # Sadece boşluk varsa olduğu gibi bırak
                translated_parts.append(part)

    return ''.join(translated_parts)

In [ ]:
# JSON içindeki tüm metinleri rekürsif olarak çevir
def translate_json(data):
    if isinstance(data, dict):
        return {k: translate_json(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [translate_json(item) for item in data]
    elif isinstance(data, str):
        return translate_text_with_html(data)
    else:
        return data

In [ ]:
input_path = "tr.json"
output_path = "translated_eng_9.json"

In [ ]:
# JSON dosyasını oku, çevir ve kaydet
with open(input_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

translated_data = translate_json(original_data)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(translated_data, f, ensure_ascii=False, indent=2)

print(f"Çeviri tamamlandı")

# Çeviri süresi : 1 dakika 58 saniye
# Kullanılan GPU A100
# Kullanılan Sistem RAM : 5.8 GB
# Kullanılan GPU RAM : 13 GB

# Sonuç

Bu çalışmada, JSON tabanlı kullanıcı arayüzü metinlerinin çok dilli çevirisi için farklı açık kaynak modelleri denendi. Denenen modeller arasında Facebook’un NLLB-200 (3.3B), M2M100 (1.2B ve 12B), Helsinki-NLP’nin Opus-MT (tr-en) ve Argos Translate yer almaktadır. İlk beş modelde başarılı sonuçlar alınırken, Google mT5-large modeli 40 dakikadan fazla sürmesi sebebiyle sonlandırıldı. Ayrıca, LLaMAX-13B ve Tower 13B gibi büyük modeller ise Türkçe dili desteklemediğinden dolayı çalıştırılmadı. Bu çalıştırılmayan modeller, ingilizceden başka dillere çeviri yapılırken çalıştırılıp değerlendirilebilir.



Karar Verilen Yapının Daha Gelişmiş Github Linki :

Türkçe - İngilizce Çeviri Modelleri Karşılaştırma Raporu Linki :